In [11]:
import json
import requests as req
import pandas as pd
import numpy as np

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, DateTime
from sqlalchemy.orm import Session

from geopy.geocoders import Nominatim

In [2]:
# page = 1
# url = 'https://api.usaspending.gov/api/v1/awards/?limit=500&page=' + str(page)
url = 'https://api.usaspending.gov/api/v1/awards/?limit=500&page=1'

params = {
    "filters": [
      {
        "field": "date_signed",
        "operation": "greater_than_or_equal",
        "value": "2017-01-01"
      },
      {
        "field": "date_signed",
        "operation": "less_than",
        "value": "2017-12-31"
      }, 
      {
        "field": "category",
        "operation": "equals",
        "value": "contract"
      },
      {
        "field": "awarding_agency__toptier_agency__cgac_code",
        "operation": "in",
        "value": ["012", "069", "014"]
      },
        {
        "field": "place_of_performance__country_name",
        "operation": "equals",
        "value": "UNITED STATES"
        }]
}

In [16]:
us_data = req.post(url , json=params).json()
print(json.dumps(us_data['page_metadata'], indent=4, sort_keys=True))

{
    "count": 84704,
    "current": "https://api.usaspending.gov/api/v1/awards/?limit=500&page=1",
    "has_next_page": true,
    "has_previous_page": false,
    "next": "https://api.usaspending.gov/api/v1/awards/?limit=500&page=2",
    "page": 1,
    "previous": null
}


In [17]:
print(json.dumps(us_data['results'][0], indent=4, sort_keys=True))

{
    "awarding_agency": {
        "id": 732,
        "office_agency": null,
        "subtier_agency": {
            "abbreviation": "OST",
            "name": "Immediate Office of the Secretary of Transportation",
            "subtier_code": "6901"
        },
        "toptier_agency": {
            "abbreviation": "DOT",
            "cgac_code": "069",
            "fpds_code": "6900",
            "name": "Department of Transportation"
        },
        "toptier_flag": false
    },
    "category": "contract",
    "certified_date": null,
    "date_signed": "2017-04-06",
    "date_signed__fy": 2017,
    "description": "IGF::OT::IGF ATEPS SBSA UNMANNED AIRCRAFT SYSTEMS (UAS)",
    "fain": null,
    "funding_agency": {
        "id": 732,
        "office_agency": null,
        "subtier_agency": {
            "abbreviation": "OST",
            "name": "Immediate Office of the Secretary of Transportation",
            "subtier_code": "6901"
        },
        "toptier_agency": {
            

In [20]:
us_awards = us_data['results'][0]['category']
us_awards

'contract'

In [4]:
next_page = True

contract_data = []

url = 'https://api.usaspending.gov/api/v1/awards/?limit=500&page=1'
page = 1

# while next_page:
while page<3:
    
    contracts_response = req.post(url , json=params).json()
    
    contracts_list = contracts_response['results']
    
    print(f'now processing page: {contracts_response["page_metadata"]["page"]}')
    
    for contract in contracts_list:
        
        contract_dict = {}
        try:
            contract_dict['Awarding_Agency'] = contract['awarding_agency']['toptier_agency']['name']
        except:
            contract_dict['Awarding_Agency'] = np.nan
        try:    
            contract_dict['Subtier_Agency'] = contract['awarding_agency']['subtier_agency']['name']
        except:
            contract_dict['Subtier_Agency'] = np.nan
        try:
            contract_dict['Subtier_Code'] = contract['awarding_agency']['subtier_agency']['subtier_code']
        except:
            contract_dict['Subtier_Code'] = np.nan
        try:
            contract_dict['Category'] = contract['category']
        except:
            contract_dict['Category'] = np.nan
        try:
            contract_dict['POP_City'] = contract['place_of_performance']['city_name']
        except:
            contract_dict['POP_City'] = np.nan
        try:
            contract_dict['POP_State'] = contract['place_of_performance']['state_name']
        except:
            contract_dict['POP_State'] = np.nan
        try:
            contract_dict['POP_State_Code'] = contract['place_of_performance']['state_code']
        except:
            contract_dict['POP_State_Code'] = np.nan
        try:
            contract_dict['POP_Zip'] = contract['place_of_performance']['zip5']
        except:
            contract_dict['POP_Zip'] = np.nan
        try:
            contract_dict['Recipient_Name'] = contract['recipient']['recipient_name']
        except:
            contract_dict['Recipient_Name'] = np.nan
        try:
            contract_dict['Total_Obligation'] = contract['total_obligation']
        except:
            contract_dict['Total_Obligation'] = np.nan
        try:
            contract_dict['Description'] = contract['description']
        except:
            contract_dict['Description'] = np.nan
        try:
            contract_dict['Date_Signed'] = contract['date_signed']
        except:
            contract_dict['Date_Signed'] = np.nan
            
        contract_data.append(contract_dict)
    
    next_page = contracts_response['page_metadata']['has_next_page']
    url = contracts_response['page_metadata']['next']
    page += 1

now processing page: 1
now processing page: 2


In [10]:
contract_df = pd.DataFrame(contract_data)

contract_df['Latitude'] = ""
contract_df['Longitude'] = ""
contract_df.head(10)

,Awarding_Agency,Category,Date_Signed,Description,POP_City,POP_State,POP_State_Code,POP_Zip,Recipient_Name,Subtier_Agency,Subtier_Code,Total_Obligation,Latitude,Longitude
0,Department of Transportation,contract,2017-04-06,IGF::OT::IGF ATEPS SBSA UNMANNED AIRCRAFT SYST...,WASHINGTON,DISTRICT OF COLUMBIA,DC,20591,EVANS INCORPORATED,Immediate Office of the Secretary of Transport...,6901,745068.00,,
1,Department of Transportation,contract,2017-06-01,THE PURPOSE OF THIS TASK ORDER IS TO PROVIDE F...,BALTIMORE,MARYLAND,MD,21230,MARYLAND MARITIME INC,Maritime Administration,6938,520169.00,,
2,Department of Transportation,contract,2017-06-01,THE PURPOSE OF THIS TASK ORDER IS TO PROVIDE F...,BALTIMORE,MARYLAND,MD,21230,MARYLAND MARITIME INC,Maritime Administration,6938,662420.00,,
3,Department of Transportation,contract,2017-02-01,IGF::OT::IGF DTFAAC-16-D-00005 LINE ITEM # 000...,ORLANDO,FLORIDA,FL,32822,"SIMCOM INTERNATIONAL, INC.",Federal Aviation Administration,6920,39520.00,,
4,Department of Transportation,contract,2017-01-17,IGF::OT::IGF:: - AIRCRAFT ICING WITH 4 DEMO FL...,MOJAVE,CALIFORNIA,CA,93501,NATIONAL TEST PILOT SCHOOL INC,Federal Aviation Administration,6920,29950.00,,
5,Department of Transportation,contract,2017-02-10,IGF::OT::IGF DTFAAC-16-D-00046 REQUEST FOR DE...,FLUSHING,NEW YORK,NY,11371,FLIGHTSAFETY INTERNATIONAL INC,Federal Aviation Administration,6920,59200.00,,
6,Department of Transportation,contract,2017-06-22,IGF::OT::IGF REQUEST FOR TASK ORDER DTFAAC-16-...,DALLAS,TEXAS,TX,75261,"CAE SIMUFLITE, INC",Federal Aviation Administration,6920,7693.00,,
7,Department of Transportation,contract,2017-05-11,N/A; ASSETS IN SUPPORT OF THE NAS,SAINT PETERSBURG,FLORIDA,FL,33710,"SENSOR SYSTEMS, L.L.C.",Federal Aviation Administration,6920,211847.00,,
8,Department of Transportation,contract,2017-04-05,COMPETENT TOWER CLIMBER&RESCUE TRAININGIGF::OT...,OKLAHOMA CITY,OKLAHOMA,OK,73169,ONESIMUS DEFENSE LLC,Federal Aviation Administration,6920,13989.00,,
9,Department of Transportation,contract,2017-09-27,N/A RECEIVER/ TRANSMITTER,WASHINGTON,DISTRICT OF COLUMBIA,DC,20591,NEW BEDFORD PANORAMEX CORP.,Federal Aviation Administration,6920,490200.00,,


In [16]:
geolocator = Nominatim()
location = geolocator.geocode("175 5th Avenue NYC")
print((location.latitude, location.longitude))

GeocoderTimedOut: Service timed out

In [15]:
for index, row in contract_df.iterrows():
    
    location = geolocator.geocode(row['POP_Zip'])
    
    row['Latitude'] = location.latitude
    row['Longitude'] = location.longitude

GeocoderTimedOut: Service timed out

In [ ]:
# Establish Connection to database
engine = create_engine(f'sqlite:///app/non-static/gov_awards.sqlite')

# Create both the Stations and Measurements tables within the database
Base.metadata.create_all(engine)

# create session object
session = Session(bind=engine)

In [ ]:
awards_list = []

for us_award in us_awards:
    
    award = Awards(
        awarding_agency = us_award['awarding_agency']['toptier_agency']['name'],
        date_signed = us_award['date_signed'],
        recipient_name = us_award['recipient']['recipient_name'],
        recipient_zip = us_award['recipient']['location']['zip5'],
        total_obligation = us_award['total_obligation']
    )
    
    awards_list.append(award)

In [ ]:
# add stations instances to hawaii.sqlite database
session.bulk_save_objects(awards_list)

# commit changes
session.commit()

In [ ]:
awards = (session.query(Awards).limit(5))

for award in awards:
    
    print(award.total_obligation)

In [ ]:
# create awards class
class Awards(Base):
    __tablename__ = 'awards'
    id = Column(Integer, primary_key=True)
    awarding_agency = Column(String(255))
    date_signed = Column(String(255))
    recipient_name = Column(String(255))
    recipient_zip = Column(String(255))
    total_obligation = Column(Float)